In [ ]:
#pip install mapclassify

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import seaborn as sns
import geopandas as gpd
import mapclassify
import joblib
import os

### Atributos Climáticos

In [ ]:
tabela = pd.read_csv('../dados_coletados/tabela_modelagem.csv', na_values = '...')
historico = pd.read_csv('../dados_coletados/historico.csv', na_values = '...')
p2_otimista = pd.read_csv('../dados_coletados/CE0007PE0002.csv', na_values = '...')
p2_intenso = pd.read_csv('../dados_coletados/CE0009PE0002.csv', na_values = '...')
p3_otimista = pd.read_csv('../dados_coletados/CE0007PE0003.csv', na_values = '...')
p3_intenso = pd.read_csv('../dados_coletados/CE0009PE0003.csv', na_values = '...')

In [ ]:
# definir lista com ordem das colunas para padronização
atributos = tabela.columns[11:]
ordem = list(p2_otimista.columns[0:3]) + list(atributos)

In [ ]:
# padronizar ordem das colunas
p2_otimista = p2_otimista[ordem]
p2_intenso = p2_intenso[ordem]
p3_otimista = p3_otimista[ordem]
p3_intenso = p3_intenso[ordem]

In [ ]:
# separar os anos de interesse
historico_2005 = historico[historico['Ano'] == 2005]
otimista_2045 = p2_otimista[p2_otimista['Ano'] == 2045]
otimista_2070 = p2_otimista[p2_otimista['Ano'] == 2070]
otimista_2099 = p3_otimista[p3_otimista['Ano'] == 2099]
intenso_2045 = p2_intenso[p2_intenso['Ano'] == 2045]
intenso_2070 = p2_intenso[p2_intenso['Ano'] == 2070]
intenso_2099 = p3_intenso[p3_intenso['Ano'] == 2099]

In [ ]:
# verificar integridade dos dados
print(len(historico_2005))
print(len(otimista_2045))
print(len(otimista_2070))
print(len(otimista_2099))
print(len(intenso_2045))
print(len(intenso_2070))
print(len(intenso_2099))

### Padronizar os atributos climáticos

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler_reg = joblib.load('../resultados/scaler_reg.sav')
scaler_class = joblib.load('../resultados/scaler_class.sav')

In [ ]:
#Regressão
h_2005_X_reg = scaler_reg.transform(historico_2005.iloc[:,3:])
o_2045_X_reg = scaler_reg.transform(otimista_2045.iloc[:,3:])
o_2070_X_reg = scaler_reg.transform(otimista_2070.iloc[:,3:])
o_2099_X_reg = scaler_reg.transform(otimista_2099.iloc[:,3:])
i_2045_X_reg = scaler_reg.transform(intenso_2045.iloc[:,3:])
i_2070_X_reg = scaler_reg.transform(intenso_2070.iloc[:,3:])
i_2099_X_reg = scaler_reg.transform(intenso_2099.iloc[:,3:])

#Classificação
h_2005_X_class = scaler_class.transform(historico_2005.iloc[:,3:])
o_2045_X_class = scaler_class.transform(otimista_2045.iloc[:,3:])
o_2070_X_class = scaler_class.transform(otimista_2070.iloc[:,3:])
o_2099_X_class = scaler_class.transform(otimista_2099.iloc[:,3:])
i_2045_X_class = scaler_class.transform(intenso_2045.iloc[:,3:])
i_2070_X_class = scaler_class.transform(intenso_2070.iloc[:,3:])
i_2099_X_class = scaler_class.transform(intenso_2099.iloc[:,3:])

### Predizer a produção de soja

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
rf_reg = joblib.load('../resultados/rf_reg.sav')
rf_class = joblib.load('../resultados/rf_class.sav')

In [ ]:
# Regressão
h_2005_reg_alvo = rf_reg.predict(h_2005_X_reg)
o_2045_reg_alvo = rf_reg.predict(o_2045_X_reg)
o_2070_reg_alvo = rf_reg.predict(o_2070_X_reg)
o_2099_reg_alvo = rf_reg.predict(o_2099_X_reg)
i_2045_reg_alvo = rf_reg.predict(i_2045_X_reg)
i_2070_reg_alvo = rf_reg.predict(i_2070_X_reg)
i_2099_reg_alvo = rf_reg.predict(i_2099_X_reg)

# Classificação
h_2005_class_alvo = rf_class.predict(h_2005_X_class)
o_2045_class_alvo = rf_class.predict(o_2045_X_class)
o_2070_class_alvo = rf_class.predict(o_2070_X_class)
o_2099_class_alvo = rf_class.predict(o_2099_X_class)
i_2045_class_alvo = rf_class.predict(i_2045_X_class)
i_2070_class_alvo = rf_class.predict(i_2070_X_class)
i_2099_class_alvo = rf_class.predict(i_2099_X_class)

In [ ]:
# Regressão
historico_2005['alvo_reg'] = h_2005_reg_alvo
otimista_2045['alvo_reg'] = o_2045_reg_alvo
otimista_2070['alvo_reg'] = o_2070_reg_alvo
otimista_2099['alvo_reg'] = o_2099_reg_alvo
intenso_2045['alvo_reg'] = i_2045_reg_alvo
intenso_2070['alvo_reg'] = i_2070_reg_alvo
intenso_2099['alvo_reg'] = i_2099_reg_alvo

# Classificação
historico_2005['alvo_class'] = h_2005_class_alvo
otimista_2045['alvo_class'] = o_2045_class_alvo
otimista_2070['alvo_class'] = o_2070_class_alvo
otimista_2099['alvo_class'] = o_2099_class_alvo
intenso_2045['alvo_class'] = i_2045_class_alvo
intenso_2070['alvo_class'] = i_2070_class_alvo
intenso_2099['alvo_class'] = i_2099_class_alvo

In [ ]:
savedir = '../resultados/previsao'
if not os.path.exists(savedir):
    os.makedirs(savedir)
historico_2005.to_csv(savedir + '/historico_2005.csv', index = False)
otimista_2045.to_csv(savedir + '/otimista_2045.csv', index = False)
otimista_2070.to_csv(savedir + '/otimista_2070.csv', index = False)
otimista_2099.to_csv(savedir + '/otimista_2099.csv', index = False)
intenso_2045.to_csv(savedir + '/intenso_2045.csv', index = False)
intenso_2070.to_csv(savedir + '/intenso_2070.csv', index = False)
intenso_2099.to_csv(savedir + '/intenso_2099.csv', index = False)

### Método para plotagem de mapa

In [ ]:
Brasil = gpd.read_file('../datasets/BR_Municipios_2021/BR_Municipios_2021.shp')

In [ ]:
Brasil = Brasil.rename(columns={'NM_MUN': 'Municipio', 'SIGLA':'Estado'})

In [ ]:
# tipo: 'reg' para regressão, 'class' para classificação
# tabela: objeto pandas com a variável alvo e as colunas 'Municipio' e 'Estado'
# alvo: nome da coluna da tabela que contém a variável alvo
def grafico(tipo, tabela, alvo):
    # Primeiro, criar uma tabela com os shapes dos municípios e com os dados
    # da variável alvo nas linhas referentes.
    modelo_plot = pd.DataFrame({
        'plot': alvo,
        'Municipio': tabela['Municipio'],
        'Estado': tabela['Estado']
    })
    global Brasil
    modelo_plot = Brasil.merge(modelo_plot, on = ['Municipio','Estado'])
    # Depois, definir o tipo de plot para modelos de regressão ou de classificação
    if tipo == 'reg':
        classes = mapclassify.UserDefined(
            modelo_plot['plot'],
            np.linspace(
                modelo_plot['plot'].min(),
                modelo_plot['plot'].max(),
                num = 8,
                endpoint = True).tolist()
        )
        fig, ax = plt.subplots()
        ax.set_facecolor('darkgrey')
        classes.plot(
            modelo_plot,
            ax = ax,
            legend = True,
            legend_kwds = {
                'loc': 'upper right',
                'bbox_to_anchor': (1.6, 1)},
            axis_on = True,
            cmap = 'OrRd',
            border_width = 0
        )
        
    elif tipo == 'class':
        fig, ax = plt.subplots()
        ax.set_facecolor('darkgrey')
        classes = {'baixo': 0, 'medio': 1, 'alto': 2}
        modelo_plot['classe'] = modelo_plot['plot'].replace(classes)
        modelo_plot.plot(
            column = 'classe',
            categorical = True,
            cmap = 'OrRd',
            ax = ax,
            legend = True,
            legend_kwds = {
                'loc': 'upper right',
                'bbox_to_anchor': (1.4, 1)}
            )
        leg = ax.get_legend()
        for text, label in zip(leg.get_texts(), classes.keys()):
            text.set_text(label)

                        
    else: print('Tipo de plot errado. Selecione \
                "reg" para modelo de regressão ou "class" para modelo de classificação.')

### Histórico 2005

##### Regressão

In [ ]:
grafico('reg', historico_2005, h_2005_reg_alvo**2)
plt.savefig('../resultados/previsao/historico_2005_reg.png', dpi = 300, bbox_inches = 'tight')

##### Classificação

In [ ]:
grafico('class', historico_2005, h_2005_class_alvo)
plt.savefig('../resultados/previsao/historico_2005_class.png', dpi = 300, bbox_inches = 'tight')

### Cenário otimista

##### Regressão

In [ ]:
grafico('reg', otimista_2045, o_2045_reg_alvo**2)
plt.savefig('../resultados/previsao/otimista_2045_reg.png', dpi = 300, bbox_inches = 'tight')

In [ ]:
grafico('reg', otimista_2070, o_2070_reg_alvo**2)
plt.savefig('../resultados/previsao/otimista_2070_reg.png', dpi = 300, bbox_inches = 'tight')

In [ ]:
grafico('reg', otimista_2099, o_2099_reg_alvo**2)
plt.savefig('../resultados/previsao/otimista_2099_reg.png', dpi = 300, bbox_inches = 'tight')

##### Classificação

In [ ]:
grafico('class', otimista_2045, o_2045_class_alvo)
plt.savefig('../resultados/previsao/otimista_2045_class.png', dpi = 300, bbox_inches = 'tight')

In [ ]:
grafico('class', otimista_2070, o_2070_class_alvo)
plt.savefig('../resultados/previsao/otimista_2070_class.png', dpi = 300, bbox_inches = 'tight')

In [ ]:
grafico('class', otimista_2099, o_2099_class_alvo)
plt.savefig('../resultados/previsao/otimista_2099_class.png', dpi = 300, bbox_inches = 'tight')

### Cenário Intenso

##### Regressão

In [ ]:
grafico('reg', intenso_2045, i_2045_reg_alvo**2)
plt.savefig('../resultados/previsao/intenso_2045_reg.png', dpi = 300, bbox_inches = 'tight')

In [ ]:
grafico('reg', intenso_2070, i_2070_reg_alvo**2)
plt.savefig('../resultados/previsao/intenso_2070_reg.png', dpi = 300, bbox_inches = 'tight')

In [ ]:
grafico('reg', intenso_2099, i_2099_reg_alvo**2)
plt.savefig('../resultados/previsao/intenso_2099_reg.png', dpi = 300, bbox_inches = 'tight')

##### Classificação

In [ ]:
grafico('class', intenso_2045, i_2045_class_alvo)
plt.savefig('../resultados/previsao/intenso_2045_class.png', dpi = 300, bbox_inches = 'tight')

In [ ]:
grafico('class', intenso_2070, i_2070_class_alvo)
plt.savefig('../resultados/previsao/intenso_2070_class.png', dpi = 300, bbox_inches = 'tight')

In [ ]:
grafico('class', intenso_2099, i_2099_class_alvo)
plt.savefig('../resultados/previsao/intenso_2099_class.png', dpi = 300, bbox_inches = 'tight')

### Comparar resutados entre cenários

##### Carregar tabelas

In [ ]:
h_2005 = pd.read_csv('../resultados/previsao/historico_2005.csv')
o_2045 = pd.read_csv('../resultados/previsao/otimista_2045.csv')
o_2070 = pd.read_csv('../resultados/previsao/otimista_2070.csv')
o_2099 = pd.read_csv('../resultados/previsao/otimista_2099.csv')
i_2045 = pd.read_csv('../resultados/previsao/intenso_2045.csv')
i_2070 = pd.read_csv('../resultados/previsao/intenso_2070.csv')
i_2099 = pd.read_csv('../resultados/previsao/intenso_2099.csv')

##### Regressão

###### Histórico

In [ ]:
print('Variância:', h_2005['alvo_reg'].var())
print('Média:', h_2005['alvo_reg'].mean())

###### Otimista

In [ ]:
display(o_2045['alvo_reg'].quantile(np.arange(0, 1.1, 0.25)))
print('Variância:', o_2045['alvo_reg'].var())
print('Média:', o_2045['alvo_reg'].mean())

In [ ]:
display(o_2070['alvo_reg'].quantile(np.arange(0, 1.1, 0.25)))
print('Variância:', o_2070['alvo_reg'].var())
print('Média:', o_2070['alvo_reg'].mean())

In [ ]:
display(o_2099['alvo_reg'].quantile(np.arange(0, 1.1, 0.25)))
print('Variância:', o_2099['alvo_reg'].var())
print('Média:', o_2099['alvo_reg'].mean())

###### Intenso

In [ ]:
display(i_2045['alvo_reg'].quantile(np.arange(0, 1.1, 0.25)))
print('Variância:', i_2045['alvo_reg'].var())
print('Média:', i_2045['alvo_reg'].mean())

In [ ]:
display(i_2070['alvo_reg'].quantile(np.arange(0, 1.1, 0.25)))
print('Variância:', i_2070['alvo_reg'].var())
print('Média:', i_2070['alvo_reg'].mean())

In [ ]:
display(i_2099['alvo_reg'].quantile(np.arange(0, 1.1, 0.25)))
print('Variância:', i_2099['alvo_reg'].var())
print('Média:', i_2099['alvo_reg'].mean())

##### Classificação

###### Histórico

In [ ]:
display(h_2005['alvo_class'].value_counts())

###### Otimista

In [ ]:
display(o_2045['alvo_class'].value_counts())

In [ ]:
display(o_2070['alvo_class'].value_counts())

In [ ]:
display(o_2099['alvo_class'].value_counts())

###### Intenso

In [ ]:
display(i_2045['alvo_class'].value_counts())

In [ ]:
display(i_2070['alvo_class'].value_counts())

In [ ]:
display(i_2099['alvo_class'].value_counts())

### Gráficos de tendência

##### Regressão

In [ ]:
reg = pd.DataFrame({
    'Ano':['2005','2045','2070','2099',
           '2005','2045', '2070', '2099'],
    'Cenário':['Otimista','Otimista','Otimista','Otimista',
              'Intenso','Intenso','Intenso','Intenso'],
    'Variância':[47.34958359396894**2, 9.297911717617247**2, 10.028279342645419**2, 2.6533585345046173**2,
                47.34958359396894*22, 10.617220036317025**2, 11.51709086998397**2, 6.956099726355543**2],
    'Média':[42.99386396320996**2, 44.75958290864603**2, 45.10285643738353**2, 42.755611686427144**2,
            42.99386396320996**2, 44.562056197096545**2, 44.86480158969739**2, 43.3775014581854**2]
    })

In [ ]:
reg_media = reg.pivot('Ano', 'Cenário', 'Média')
reg_media

In [ ]:
sns.set_theme(style = 'ticks', rc = {'axes.facecolor':'#C6C6C6'})
sns.lineplot(data = reg_media,
            markers = False, dashes = True, linewidth = 3)
plt.legend(bbox_to_anchor = (1.05, 1), loc = 2, borderaxespad = 0.1)
sns.despine()
plt.xlabel('Ano', size = 12)
plt.ylabel('Produtividade média (kg/ha)', size = 14)
plt.savefig('../resultados/previsao/reg.png', dpi = 300, bbox_inches = 'tight')

##### Classificação

In [ ]:
o_class = pd.DataFrame({
    'Ano':['2005','2005','2005','2045','2045','2045','2070','2070','2070','2099','2099','2099'],
    'Classe':['baixo','medio','alto','baixo','medio','alto','baixo','medio','alto','baixo','medio','alto'],
    'Municípios':[2078, 1188, 2297, 625, 1744, 3194, 529, 1790, 3244, 389, 4397, 777]
    })

i_class = pd.DataFrame({
    'Ano':['2005','2005','2005','2045','2045','2045','2070','2070','2070','2099','2099','2099'],
    'Classe':['baixo','medio','alto','baixo','medio','alto','baixo','medio','alto','baixo','medio','alto'],
    'Municípios':[2078, 1188, 2297, 815, 2070, 2678, 357, 1869, 3337, 254, 3604, 1705]
    })

In [ ]:
def tendencia(dados):    
    sns.set_theme(style = 'ticks', rc = {'axes.facecolor':'#C6C6C6'})
    cores = ['#7B0004','#FEF7EC','#F68A5C']
    sns.set_palette(palette = cores)
    sns.lineplot(data = dados.pivot('Ano', 'Classe', 'Municípios'),
                 markers = False, dashes = False, linewidth = 2.5)
    plt.legend(bbox_to_anchor = (1.05, 1), loc = 2, borderaxespad = 0.1)
    sns.despine()
    plt.xlabel('Ano', size = 12)
    plt.ylabel('Número de municípios', size = 14)

In [ ]:
tendencia(o_class)
plt.title('Cenário Otimista', size = 14)
plt.savefig('../resultados/previsao/otimista_class.png', dpi = 300, bbox_inches = 'tight')

In [ ]:
tendencia(i_class)
plt.title('Cenário Intenso', size = 14)
plt.savefig('../resultados/previsao/intenso_class.png', dpi = 300, bbox_inches = 'tight')